# Teoria uczenia maszynowego - Projekt zaliczeniowy

- Daniel Zdancewicz [indeks: 145317]

## Temat:

Wpływ dodawania szumu na trafność algorytmów uczenia. 

## Opis:

Celem projektu jest zbadanie wpływu dodawania szumu na trafność algorytmów uczenia maszynowego.
Szum jest dodawany do zbioru uczącego poprzez:

- dodanie dodatkowych cech wejściowych z szumem.
- zaszumienie istniejących cech.
- zaszumienie wartości na wyjściu.

## Plan realizacji i wybory projektowe:

1. Wybór zbioru danych:
    - zbiór danych "Iris" dostępny w bibliotece sklearn.datasets
    - zbiór danych "Wine" dostępny w bibliotece sklearn.datasets
2. Implementacja algorytmów uczenia maszynowego
    - Wykorzystano algorytmy: SVM, KNN, Random Forest, oraz prostą sieć neuronową MLP
    - Zaimplementowano funkcję, która zwraca trafność klasyfikacji dla zbioru danych
3. Implementacja sposobów dodawania szumu
    - Dodawanie szumu nowych cech wejściowych ze stałym szumem
    - Dodawanie szumu nowych cech wejściowych ze skorelowanym szumem
    - Dodawanie szumu do istniejących cech wejściowych ze stałym szumem
    - Dodawanie szumu do istniejących cech wejściowych ze skorelowanym szumem
    - Dodawanie szumu na wyjściu (losowe zmiany etykiet klas w części obiektów)
4. Zbadanie wpływu dodawania szumu na trafność algorytmów uczenia maszynowego
    - Zaimplementowano funkcję, która dodaje szum do zbioru danych, wewnątrz obrębu danych uczących, ale nie testowych.
    - Zbadano wpływu dodania szumu na trafność klasyfikacji dla różnych algorytmów uczenia maszynowego
5. Eksperymenty
6. Wyniki
7. Przedstawienie wniosków

### Założenia o wynikach:
- Dodanie szumu do zbioru danych prawdopodobnie obniży trafność klasyfikacji.
- Dodanie szumu do wyjść bardzo prawdopodobnie obniży trafność klasyfikacji ze względu na niepoprawność klas.
- Prawdopodobnie trafność klasyfikacji może zostać przywrócona poprzez zastosowanie odpowiednich technik regularyzacji, np PCA, L1, L2.

## Importy bibliotek i modułów

In [1]:
from analysis import visualize_correlation, visualize_metrics
from experiments import summarize_results, run_experiments, create_model_descriptors
from datasets.utils import load_datasets, create_noisy_datasets
from datasets import DatasetNoise


## Wczytanie zbiorów danych

In [2]:
iris, wine = load_datasets()


### Zbiór danych "Iris"

Zbiór składający się z 150 próbek trzech gatunków irysów (Iris setosa, Iris virginica, Iris versicolor). 

Dla każdego irysa mamy 4 cechy: długość i szerokość płatków oraz długość i szerokość działki kielicha.

Wszystkie te wartości są wartościami liczbowymi, założyłem dodatkową walidację wewnątrz klasy `IrisDataset` sprawdzającą czy wartości są liczbami.

Zbiór ten jest dosyć mały i ma mało cech, służył mi jako zbiór do testów i weryfikacji poprawności implementacji.

In [3]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Zbiór danych "Wine"

Zbiór składający się z 178 próbek trzech gatunków win (klasy 0, 1, 2).

Dla każdego wina mamy 13 cech:
- Alcohol, Malic acid, Ash, Alcalinity of ash, Magnesium, Total phenols, Flavanoids, Nonflavanoid phenols, Proanthocyanins, Color intensity, Hue, OD280/OD315 of diluted wines, Proline

Wszystkie są również wartościami liczbowymi, i również założyłem dodatkową walidację wewnątrz klasy `WineDataset` sprawdzającą czy wartości są liczbami.

Ten zbiór jest większy i ma więcej cech przez co pozwala na bardziej złożone eksperymenty.

In [4]:
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


## Algorytmy uczenia maszynowego

Skorzystałem z gotowych implementacji algorytmów dostępnych w bibliotece `scikit-learn`.

Wykorzystuje z niej algorytmy:
- SVM, KNN, RandomForest oraz prostą sieć MLP.
- KNN dałem w 3 wariantach z różną ilością sąsiadów ( 2, 3, 4 ).

Implementacja algorytmów znajduje się w metodzie `create_model_descriptors`, która zwraca fabryki modeli z odpowiednimi labelkami.


In [5]:
descriptors = create_model_descriptors()

## Algorytmy zaszumienia danych

Do zaszumienia danych utworzyłem klasę `DatasetNoise` która pozwala na dodanie szumu do zbioru danych za pomocą 4 algorytmów. Podczas eksperymentów szumowanie danych odbywa się wewnątrz zbioru uczącego, ale nie testowego.


### Pomocnicze funkcje do wizualizacji różnic

In [6]:
from raport.display import render_difference, pick_first_column_id, pick_second_column_id

In [7]:
column_id = pick_first_column_id(iris)
clean = iris.head()

## Dodanie stałego szumu do kolumny

Metody `add_static_noise/add_static_noises` - dodają szum do kolumny na podstawie różnicy maksymalnej i minimalnej wartości pomnożonej przez współczynnik skalowania.

In [8]:
noisy_static = DatasetNoise(iris).add_static_noise(column_id, 0.1).build().head()

render_difference(clean, noisy_static, column_id=column_id, title="Stały szum w skali 0.1")


,sepal_length
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0
,sepal_length
0,0.36
1,0.36
2,0.36
3,0.36


## Dodanie losowego szumu do kolumny

Metody `add_random_noise/add_random_noises` - dodają losowy szum do kolumny na podstawie różnicy maksymalnej i minimalnej wartości pomnożonej przez współczynnik skalowania próbkowany z rand(scale_min, scale_max).

In [9]:
noisy_random = DatasetNoise(iris).add_random_noise(column_id, (0.0, 0.2)).build().head()

render_difference(clean, noisy_random, column_id=column_id, title="Losowy szum w skali (0.0, 0.2)")

,sepal_length
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0
,sepal_length
0,0.556533
1,0.468886
2,0.430693
3,0.310957


## Dodanie stałego skorelowanego szumu do kolumny

Metody `add_static_correlated_noise/add_static_correlated_noises` - dodają skorelowany szum do kolumny na podstawie różnicy maksymalnej i minimalnej wartości pomnożonej przez normy wskazanej kolumny.

In [10]:
first_column_id = pick_first_column_id(iris)
second_column_id = pick_second_column_id(iris)
column_ids = [first_column_id, second_column_id]

noisy_correlated_static = (
  DatasetNoise(iris)
  .add_static_correlated_noise(first_column_id, second_column_id)
  .build()
  .head()
)

render_difference(clean, noisy_correlated_static, column_ids=column_ids, title="Stały skorelowany szum")

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
,sepal_length,sepal_width
0,2.25,0.0
1,1.50,0.0
2,1.80,0.0
3,1.65,0.0


## Dodanie losowego skorelowanego szumu do kolumny

Metody `add_random_correlated_noise/add_random_correlated_noises` - dodają skorelowany szum do kolumny na podstawie różnicy maksymalnej i minimalnej wartości pomnożonej przez normy wskazanej kolumny próbkowany z rand(scale_min, scale_max).


In [11]:
noisy_correlated_random = (
  DatasetNoise(iris)
  .add_random_correlated_noise(first_column_id, second_column_id, (0.0, 0.2))
  .build()
  .head()
)

render_difference(clean, noisy_correlated_random, column_ids=column_ids, title="Losowy skorelowany szum w skali (0.0, 0.2)")

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
,sepal_length,sepal_width
0,3.399787,0.0
1,0.242878,0.0
2,2.760595,0.0
3,0.500071,0.0


## Dodanie szumu do wyjścia

Metoda `shuffle` - Miesza wskazany procent etykiet klas w zbiorze danych w obrębie wskazanej kolumny.

In [12]:
def pick_last_column_id(dataset):
    return dataset.columns[-1]

target_column_id = pick_last_column_id(iris)

noisy_output = DatasetNoise(iris).shuffle(target_column_id, percentage=1.0).build().head()

render_difference(clean, noisy_output, column_id=target_column_id, title="Szum na wyjściu po przemieszaniu etykiet")

,target
0,0
1,0
2,0
3,0
4,0
,target
0,2
1,0
2,1
3,0


## Eksperymenty

Eksperymenty polegały na zbadaniu wpływu dodawania szumu na różnych metrykach dla różnych algorytmów uczenia maszynowego.

Implementacja przebiegu eksperymentu jest opisany wewnątrz metody `run_experiments`, a sposób jego podsumowania wewnątrz metody `summarize_results`. Podobnie metody wizualizacji są wewnątrz metody `visualize_metrics` oraz `visualize_correlation`.

## Wyniki

## Wizualizacja wyników

## Wnioski